In [130]:
from bs4 import BeautifulSoup 
import requests
import urllib
import re
import os
from urllib.parse import urljoin
from lxml import etree
import math
import json
import pandas as pd

# 1 | DEFINING FUNCTION

## 1.1 | REQUESTING FUNCTION

    Em xem trên mạng người ta chỉ có cái web "scrapeops" cho mình fake user agents. 
    Nhưng mà nhiều web vẫn không fake được vì còn nhiều cái khác block mình

In [131]:
def get_agents():
    response = requests.get(
                        url='https://headers.scrapeops.io/v1/user-agents',
                        params={
                            'api_key': 'c77ada02-6767-4b0e-bd5d-38aea43c9431',
                            'num_results': '2'}
                        )
    agents = response.json()['result']
    return agents

In [132]:
def extract(url, agents):
    for agent in agents:
        headers = {'User-Agent': agent}
        r = urllib.request.Request(url, headers= headers)
        html = urllib.request.urlopen(r).read()
        soup = BeautifulSoup(html, 'html.parser')
        return soup

## 1.2 | FUNCTION THAT EXTRACTS DATA FROM 1 URL 

    2 function chỗ này dùng để:
    - get_job_soups: 
        Mục đích: lấy tất cả URL mà thông tin chi tiết của các job được để
        Arguments:
            soup: soup object của page được hiển thị sau khi mình search lần đầu trên web, là base url
            website: mỗi web lấy mỗi khác nên em dùng arg website để tách cách xử lý input
        Output: các soup objects của những page thông tin chi tiết của các job
    - get_infor_from_job_soups:
        Mục đích: lấy tất cả thông tin cần thiết từ page thông tin chi tiết của mỗi job post
        Arguments:
            job_soups: output của get_job_soups, là 1 list các soup objects của các page job
            website: mỗi web lấy mỗi khác nên em dùng arg website để tách cách xử lý input
        Output: thông tin chi tiết của từng job, dưới dạng json
    

In [133]:
# get link to job items 
def get_job_soups(soup, domain_url, website):
    if website == 'ITVIEC':
        divs = soup.find_all('div', class_ = 'job-card ipt-2 d-flex flex-column border-radius-large position-relative bg-white')
        job_links = [div.select_one('h3.imt-3 > a')['href'] for div in divs]
        job_links = [urljoin(domain_url, job_link)for job_link in job_links]
        job_soups = [extract(url=job_link, agents=get_agents()) for job_link in job_links]
    elif website == 'VIECLAM24H':
        divs = soup.find_all('div', class_ = 'relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
        job_links = [div.select_one("a")['href'] for div in divs]
        job_links = [urljoin(domain_url, job_link)for job_link in job_links]
        job_soups = [extract(url=job_link, agents=get_agents()) for job_link in job_links]
    elif website == 'JOBSGO':
        divs = soup.find_all('article', class_ = 'article')
        job_links = [div.select_one('a')['href'] for div in divs]
        job_soups = [extract(url=job_link, agents=get_agents()) for job_link in job_links]
    elif website == 'JOBOKO':
        job_links = [s['href'] for s in soup.select("div[class='item-head'] > h2 > a")]
        job_links = [urljoin(domain_url, job_link)for job_link in job_links]
        job_soups = [extract(url=job_link, agents=get_agents()) for job_link in job_links]
    return job_soups

In [134]:
# extract data from job page
def get_infor_from_job_soups(job_soups, website):
    if website == 'ITVIEC':
        data_list = []
        for soup in job_soups:
            data = {
                'JobTitle': soup.select_one('div.job-header-info > h1').text.strip(),
                'Location': soup.find_all('span', class_ ='normal-text text-rich-grey')[0].text.strip(),
                'PostDate': soup.select_one("div[class='d-inline-block text-dark-grey preview-header-item'] > span[class='normal-text text-rich-grey']").text.strip(),
                'JobDescription': [s.text.strip() for s in soup.select("div[class='imy-5 paragraph']")],
                'Tools': [s.text.strip() for s in soup.select("div[class='imt-2'] > a > div")]
            }
            data_list.append(data)
    elif website == 'VIECLAM24H':
        data_list = []
        for soup in job_soups:
            data = {
                'JobTitle': soup.select_one("div[class='md:ml-7 w-full'] > h1").text.strip(),
                'Location': soup.select_one("div[class='ml-3 text-14 md:flex pt-0 md:pt-[5px]'] > p > a")['title'],
                'PostDate': soup.select_one("div[class='ml-3'] > p[class='text-14']").text.strip(),
                'JobDescription': [s.text.strip() for s in soup.select("div[class='jsx-d84db6a84feb175e mb-2 text-14 break-words text-se-neutral-80 text-description']")],
                'Level': re.findall("Cấp bậc(.*)", string=soup.select_one("i[class='w-[32px] h-[32px] min-w-[32px] flex items-center justify-center rounded-full bg-[#EDE9FE] text-[#8B5CF6] text-20 svicon-medal'] + div").text.strip()),
                'ExperienceRequirement': re.findall("Yêu cầu kinh nghiệm(.*)", string=soup.select_one("i[class='w-[32px] h-[32px] min-w-[32px] flex items-center justify-center rounded-full bg-[#EDE9FE] text-[#8B5CF6] text-20 svicon-experience-user'] + div").text.strip()),
                'SalaryRange': re.findall("Mức lương : (.*)",string=soup.select_one("div[class='flex items-start min-w-[250px] mb-4']>div[class='ml-3 text-14 md:flex pt-0 md:pt-[5px]']").text.strip()),
                'CompanyName': soup.select_one("div[class='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6'] > a > h3").text.strip(),
            }
            data_list.append(data)
    elif website == 'JOBSGO':
        data_list = []
        for soup in job_soups:
            data = {
                'JobTitle': soup.select_one("div[class='media-body-2'] > h1").text.strip(),
                'Location': soup.select_one("div[class='content-group'] > div[class='box-jobs-info'] > div p").text.strip(),
                'PostDate_Level_ExperienceRequirement': "\n".join([s.text.strip() for s in soup.select("div[class='col-sm-4 col-xs-6'] > p")]),
                'JobDescription': [s.text.strip() for s in soup.find_all("div", class_="clearfix")],
                'SalaryRange': re.findall("Mức lương (.*)", string = "\n".join([s.text.strip() for s in soup.select("div[class='media-body-2'] li")])),
                'CompanyName': soup.select_one("div[class='profile-cover'] div[class='media-body'] > h2 > a").text.strip() if soup.select_one("div[class='profile-cover'] div[class='media-body'] > h2 > a") else ""
            }
            data_list.append(data)
    elif website == 'JOBOKO':
        data_list = []
        for soup in job_soups:
            data = {
                'JobTitle': soup.select_one("h2[class='fz-20 fw-bold letter-spacing-1 text-uppercase c-green nw-company-hero__title']").text.strip(),
                'Location': re.findall("Địa điểm làm việc: (.*)", string=soup.select_one("div[class='nw-company-hero__address'] > span").text.strip()),
                'JobDescription': [s.text.strip() for s in soup.select("div[class='fz-15 block-text c-text-2'] > div[class='text-justify']")],
                'ExperienceRequirement': re.findall("Kinh nghiệm: (.*) năm", "\n".join([s.text.strip() for s in soup.select("div[class='fz-15 letter-spacing-1 item-content']")])),
                'SalaryRange': re.findall("Mức lương: (.*)", soup.select_one("div[class='col-12'] div[class='fz-15 letter-spacing-1 item-content']").text.strip()), 
                'Level': re.findall("Chức vụ: (.*)" , "\n".join([s.text.strip() for s in soup.select("div[class='fz-15 letter-spacing-1 item-content']")])),
                'CompanyName': soup.select_one("div[class='nw-company-hero__info'] > a").text.strip() if soup.select_one("div[class='nw-company-hero__info'] > a") else soup.select_one("div[class='nw-company-hero__info'] > div[class='fz-16 letter-spacing-1 text-capitalize c-text-2 nw-company-hero__text']").text.strip()
            }
            data_list.append(data)
    return data_list

    function "extract_data_from_curr_url" là để gộp 2 function phía trên vào chung để thực hiện cùng 1 lần
    process:
    - nó lấy url đầu tiên mà mình search ("current url")
    - convert page thành soup object
    - lấy soup objects của các page thông tin chi tiết của các job
    - lấy thông tin chi tiết của các job post
    - trả về json

In [135]:
# combine get_job_soups > get_infor_from_job_soups
def extract_data_from_curr_url(domain_url, current_url, website):
    soup = extract(url=current_url, agents = get_agents())
    job_soups = get_job_soups(soup, domain_url, website)
    data_list = get_infor_from_job_soups(job_soups, website)
    return data_list

## 1.3 | FUNCTION THAT GETS ALL URLS ASSOCIATED WITH 1 URL

    function "get_all_next_pages_url" dùng để lấy tất cả url liên kết với url gốc của mình, dùng để cào từ page này sang page khác
    mỗi website process khác nhau

In [136]:
# get all next pages url to enable spider
def get_all_next_pages_url(curr_soup, domain_url, website, query):
    if website == 'ITVIEC':
        all_next_pages_url = []
        while curr_soup.select_one('div[class="page next"]') is not None:
            next_url = curr_soup.select_one('div[class="page next"] >a')['href']
            full_next_url = urljoin(domain_url, next_url)
            all_next_pages_url.append(full_next_url)
            curr_soup = extract(full_next_url, agents = get_agents())
    elif website == 'VIECLAM24H':
        # round up the number of pages
        pages = math.ceil(int(curr_soup.select_one("div[class='flex items-center'] span[class='font-semibold']").text.strip()) / 30)
        all_next_pages_url = []
        for page in range(2, pages + 1): # to get correct next pages
            url = "https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page={}&q={}&sort_q=".format(str(page),query)
            all_next_pages_url.append(url)
    elif website == 'JOBSGO':
        # round up the number of pages
        pages = math.ceil(int(re.findall(".* / (.*)", string = curr_soup.select_one("div[class='sidebar-widget-title mrg-bot-15'] > span").text.strip())[0]) / 50)
        all_next_pages_url = []
        for page in range(2, pages + 1): # to get correct next pages
            url = "https://jobsgo.vn/viec-lam-{}-tai-ho-chi-minh.html?page={}".format(query, str(page))
            all_next_pages_url.append(url)
    elif website == 'JOBOKO':
        all_next_pages_url = []
        pages = math.ceil(int("".join(re.findall("[0-9]", curr_soup.select_one("li[class='nav-item'] >a>span").text))) / 10)
        true_pages = 50 if pages >= 50 else pages
        for page in range(2, true_pages+1):
            url = "https://vn.joboko.com/jobs?q={}&p={} ".format(query,page)
            all_next_pages_url.append(url)
    return all_next_pages_url

## 1.4 | FUNCTION THAT CRAWLS THROUGH ALL THOSE URLS AND GET DATA

    function "crawl_data_from_url" cào dữ liệu từ tất cả page
    nhận input là url đầu tiên, website đang cào, và query search

In [137]:
# FINAL FUNCTION THAT CRAWLS THROUGH MULTIPLE PAGES TO EXTRACT RELEVANT DATA
def crawl_data_from_url(domain_url, current_url, website, query):
    # get all pages urls
    curr_soup = extract(current_url, agents = get_agents())
    all_next_pages_url = get_all_next_pages_url(curr_soup, domain_url, website = website, query = query)
    print('got all urls!')
    print(all_next_pages_url)
    # initiate first url
    data_list_of_all_urls = extract_data_from_curr_url(domain_url, current_url = current_url, website = website)
    for page_url in all_next_pages_url:
        data_list_cur_url = extract_data_from_curr_url(domain_url, current_url = page_url, website = website)
        for data_list in data_list_cur_url:
            data_list_of_all_urls.append(data_list)
        print("extracted data from {}".format(page_url))
    return all_next_pages_url, data_list_of_all_urls

# 2 | SAMPLE RUN ON SOME WEBSITES

In [119]:
all_next_pages_url, data_list_of_all_urls = crawl_data_from_url(
    domain_url = 'https://jobsgo.vn/', 
    current_url = 'https://jobsgo.vn/viec-lam-data.html', 
    website = 'JOBSGO', 
    query = 'data'
)

got all urls!
['https://jobsgo.vn/viec-lam-data-tai-ho-chi-minh.html?page=2']
extracted data from https://jobsgo.vn/viec-lam-data-tai-ho-chi-minh.html?page=2


In [138]:
all_next_pages_url, data_list_of_all_urls = crawl_data_from_url(
    domain_url = 'https://vieclam24h.vn/', 
    current_url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?q=IT', 
    website = 'VIECLAM24H', 
    query = 'IT'
)

df_VIECLAM24H = pd.DataFrame(data_list_of_all_urls)
df_VIECLAM24H

got all urls!
['https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&q=IT&sort_q=', 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=3&q=IT&sort_q=', 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=4&q=IT&sort_q=', 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=5&q=IT&sort_q=', 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=6&q=IT&sort_q=', 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=7&q=IT&sort_q=']
extracted data from https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&q=IT&sort_q=
extracted data from https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=3&q=IT&sort_q=
extracted data from https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=4&q=IT&sort_q=
extracted data from https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=5&q=IT&sort_q=
extracted data from https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=6&q=IT&sort_q=
extracted data from https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=7&q=IT&sort_q=


,JobTitle,Location,PostDate,JobDescription,Level,ExperienceRequirement,SalaryRange,CompanyName
0,Nhân Viên IT- IT Helpdesk,Việc làm tại Bình Dương,20/12/2023,[-Xác định và giải quyết các vấn đề liên quan ...,[Chuyên viên- nhân viên],[Chưa có kinh nghiệm],[7 - 10 triệu],Công ty TNHH Liên Doanh Hasan - Dermapharm
1,Nhân Viên IT,Việc làm tại TP.HCM,25/11/2023,[- Cập nhật sản phẩm kinh doanh của công ty lê...,[Chuyên viên- nhân viên],[Dưới 1 năm],[10 - 15 triệu],Công Ty TNHH Đặng Mậu Tấn
2,"Chuyên Viên IT Helpdesk Tại Kim Mã, Hà Nội",Việc làm tại Hà Nội,15/12/2023,[- Xử lý các sự cố kỹ thuật liên quan đến hệ t...,[Chuyên viên- nhân viên],[Dưới 1 năm],[9 - 15 triệu],Công ty Tài Chính TNHH HD SAISON
3,Giáo Viên Tin Học - IT,Việc làm tại TP.HCM,11/12/2023,[- Giảng dạy theo chương trình của Bộ Giáo Dục...,[Chuyên viên- nhân viên],[Chưa có kinh nghiệm],[6 - 12 triệu],Công Ty TNHH Một Thành Viên Vi Tính Nt Đông
4,Nhân Viên IT Help Desk,Việc làm tại TP.HCM,25/11/2023,"[- Set up các phòng học vi tính, hệ thốn...",[Chuyên viên- nhân viên],[Chưa có kinh nghiệm],[7 - 15 triệu],Công Ty TNHH Tư Vấn & Đào Tạo Đại Dương
...,...,...,...,...,...,...,...,...
200,"Sale Manager (Thiết Bị Mã Vạch, Pos)",Việc làm tại TP.HCM,24/11/2023,[Chịu trách nhiệm về chỉ tiêu doanh số được gi...,[Quản lý cấp cao],[3 năm],[Thoả thuận],Công Ty Cổ Phần Thương Mại Dịch Vụ Tín Hòa
201,Nhân Viên Kỹ Thuật Dự Án - Mảng Công Nghệ Thôn...,Việc làm tại Hà Nội,24/11/2023,[- Tư vấn giải pháp cho khách hàng;- Lập báo g...,[Chuyên viên- nhân viên],[1 năm],[13 - 20 triệu],Công Ty TNHH Công Nghệ Nguyên Lộc
202,Trưởng Phòng Kỹ Thuật,Việc làm tại Hà Nội,24/11/2023,"[Quản lý, điều hành các hoạt động phòng kỹ thu...",[Quản lý cấp trung],[3 năm],[18 - 25 triệu],Công Ty TNHH Thương Mại Điện Tử Bảo Giang
203,Sóc Trăng Nhân Viên Kỹ Thuật,Việc làm tại Sóc Trăng,24/11/2023,"[- Cài đặt thiết bị, phần mềm của công ty.- Hỗ...",[Chuyên viên- nhân viên],[1 năm],[9 - 13 triệu],Công ty Cổ Phần Vacxin Việt Nam


In [151]:
df_VIECLAM24H.to_csv(r"C:\Users\Admin\airflow1412\csvfile\VIECLAM24H.csv")